# Extenstion of notebook 3 and 4
-- Actually 2.2.0 tagged version https://github.com/pyg-team/pytorch_geometric/tree/2.2.0

In [2]:
import torch
from torch import Tensor
print(torch.__version__)

import pandas as pd
from torch_geometric.data import download_url, extract_zip

import pandas as pd

from torch_geometric.data import HeteroData
import torch_geometric.transforms as T

# note it uses the wrong linkNeighborLoader (there are two). one for sampler and one for loader.
from torch_geometric.loader import LinkNeighborLoader


/Users/epedersen/.pyenv/versions/3.8.12/envs/torch/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


1.13.1


In [3]:
from torch_geometric.data import Data, HeteroData
from torch_geometric.loader import NeighborLoader
from torch_sparse import SparseTensor

# we actually doen't need this if we just use hetero data, also have graph_store interface. 
def add_date_to_graph_store(graph_store,data):
    for edge_type in train_data.edge_types: 
        edge_index = data[edge_type].edge_index
        coo = (edge_index[0], edge_index[1])
        graph_store_train.put_edge_index(edge_index = coo,
                            edge_type = edge_type,
                            layout = 'coo',
                            size = (feature_store[edge_type[0]].num_nodes, feature_store[edge_type[2]].num_nodes)
                            )

In [4]:

url = 'https://files.grouplens.org/datasets/movielens/ml-latest-small.zip'
extract_zip(download_url(url, '.'), '.')

movies_path = './ml-latest-small/movies.csv'
ratings_path = './ml-latest-small/ratings.csv'

Using existing file ml-latest-small.zip
Extracting ./ml-latest-small.zip


In [5]:
print('movies.csv:')
print('===========')
print(pd.read_csv(movies_path)[["movieId", "genres"]].head())
print()
print('ratings.csv:')
print('============')
print(pd.read_csv(ratings_path)[["userId", "movieId"]].head())

movies.csv:
   movieId                                       genres
0        1  Adventure|Animation|Children|Comedy|Fantasy
1        2                   Adventure|Children|Fantasy
2        3                               Comedy|Romance
3        4                         Comedy|Drama|Romance
4        5                                       Comedy

ratings.csv:
   userId  movieId
0       1        1
1       1        3
2       1        6
3       1       47
4       1       50


In [6]:
# Load the entire movie data frame into memory:
movies_df = pd.read_csv(movies_path, index_col='movieId')

# Split genres and convert into indicator variables:
genres = movies_df['genres'].str.get_dummies('|')
print(genres[["Action", "Adventure", "Drama", "Horror"]].head())

# Use genres as movie input features:
movie_feat = torch.from_numpy(genres.values).to(torch.float)
assert movie_feat.size() == (9742, 20)  # 20 genres in total.

         Action  Adventure  Drama  Horror
movieId                                  
1             0          1      0       0
2             0          1      0       0
3             0          0      0       0
4             0          0      1       0
5             0          0      0       0


In [7]:
# Load the entire ratings data frame into memory:
ratings_df = pd.read_csv(ratings_path)

# Create a mapping from unique user indices to range [0, num_user_nodes):
unique_user_id = ratings_df['userId'].unique()
unique_user_id = pd.DataFrame(data={
    'userId': unique_user_id,
    'mappedID': pd.RangeIndex(len(unique_user_id)),
})
print("Mapping of user IDs to consecutive values:")
print("==========================================")
print(unique_user_id.head())
print()
# Create a mapping from unique movie indices to range [0, num_movie_nodes):
unique_movie_id = ratings_df['movieId'].unique()
unique_movie_id = pd.DataFrame(data={
    'movieId': unique_movie_id,
    'mappedID': pd.RangeIndex(len(unique_movie_id)),
})
print("Mapping of movie IDs to consecutive values:")
print("===========================================")
print(unique_movie_id.head())

# Perform merge to obtain the edges from users and movies:
ratings_user_id = pd.merge(ratings_df['userId'], unique_user_id,
                            left_on='userId', right_on='userId', how='left')
ratings_user_id = torch.from_numpy(ratings_user_id['mappedID'].values)
ratings_movie_id = pd.merge(ratings_df['movieId'], unique_movie_id,
                            left_on='movieId', right_on='movieId', how='left')
ratings_movie_id = torch.from_numpy(ratings_movie_id['mappedID'].values)

# With this, we are ready to construct our `edge_index` in COO format
# following PyG semantics:
edge_index_user_to_movie = torch.stack([ratings_user_id, ratings_movie_id], dim=0)
assert edge_index_user_to_movie.size() == (2, 100836)

print()
print("Final edge indices pointing from users to movies:")
print("=================================================")
print(edge_index_user_to_movie)


Mapping of user IDs to consecutive values:
   userId  mappedID
0       1         0
1       2         1
2       3         2
3       4         3
4       5         4

Mapping of movie IDs to consecutive values:
   movieId  mappedID
0        1         0
1        3         1
2        6         2
3       47         3
4       50         4

Final edge indices pointing from users to movies:
tensor([[   0,    0,    0,  ...,  609,  609,  609],
        [   0,    1,    2,  ..., 3121, 1392, 2873]])


### Create random user links as well 
(we want to work with multiple link types)

In [8]:


import numpy as np
user_to_user = np.random.choice(unique_user_id.mappedID.values, size=(100,2), replace=True, p=None)

edge_index_user_to_user = torch.stack((torch.from_numpy(user_to_user[:,0])
                                    ,torch.from_numpy(user_to_user[:,1])),dim=0)
edge_index_user_to_user

tensor([[581,  77,  32, 314,  26, 418, 556,  31, 395, 317,  38,  50, 363, 132,
         185, 463, 600, 538, 156, 596, 527, 371,  94, 314, 140, 339, 531, 414,
         115, 344, 243, 175,  28,  56, 157, 123,  93,   1, 510, 157, 142, 294,
         585, 578, 119, 117, 512, 340, 437, 541, 241, 198, 542, 367, 263,  15,
         100, 119, 589, 603, 357, 334, 551, 438, 260, 599, 219, 177, 499, 279,
         591,  72, 530, 343, 165, 493, 154, 251, 401, 278, 421, 490, 530, 564,
         558, 142, 387,  76, 114, 498, 419, 480, 415, 192, 116, 342, 526, 420,
         608, 173],
        [474, 307, 351,  49, 393, 563, 379, 463,  84, 278, 571, 130, 341, 605,
          68,   0, 381, 551, 403, 574,  99, 110, 488, 239, 118, 146, 375,  11,
         140, 114,  10, 481,  28, 292,  30,   3,  55, 560, 428,  32, 191, 155,
         227, 408, 296, 269,  50, 527, 558, 342,  31,  49, 371, 215, 375, 133,
         211,  64, 442, 576, 523, 435, 275, 341, 364, 515, 474, 536, 561, 108,
         196, 263,  19, 219, 459

In [12]:

data = HeteroData()

## Keep data object to only transform it to the graph store after transformation
## but omit adding the feature vectors of movies.
# Save node indices:
data["user"].node_id = torch.arange(len(unique_user_id))
data["movie"].node_id = torch.arange(len(movies_df))

# Add the node features and edge indices:
# data["movie"].x = movie_feat # TODO
data["user", "rates", "movie"].edge_index = edge_index_user_to_movie  
data["user", "likes", "user"].edge_index = edge_index_user_to_user 

# We also need to make sure to add the reverse edges from movies to users
# in order to let a GNN be able to pass messages in both directions.
# We can leverage the `T.ToUndirected()` transform for this from PyG:

# TODO:
# FROM HERE https://pytorch-geometric.readthedocs.io/en/latest/modules/transforms.html
transform = T.Compose([T.ToUndirected()])
data = transform(data)          

feature_store = HeteroData()

# Add the data to the feature store
feature_store.put_tensor(movie_feat, group_name='movie', attr_name='x', index=torch.arange(len(movies_df)))
feature_store.put_tensor(torch.arange(len(movies_df)), group_name='movie', attr_name='node_id', index=torch.arange(len(movies_df)))


# TODO: Is there a way to add a feature with only id to the feature store? 
feature_store.put_tensor(torch.arange(len(unique_user_id)),group_name='user', attr_name='node_id', index=torch.arange(len(unique_user_id)))


## Delay adding graph to graph store after linksplits 
## Add graph to graph store

#graph_store = HeteroData()
# coo = (edge_index_user_to_movie[0], edge_index_user_to_movie[1])

# graph_store.put_edge_index(edge_index=coo,
#                            edge_type=('user', 'rates', 'movie'),
#                            layout='coo', size=(len(coo[0]),len(coo[1])) )

# # put reverse index
# graph_store.put_edge_index(edge_index=coo[::-1],
#                            edge_type=('movie', 'rev_rates', 'user'),
#                            layout='coo', size=(len(coo[1]),len(coo[0])) )


print(data)

assert data.node_types == ["user", "movie"]
assert data.edge_types == [("user", "rates", "movie"),
                           ("user","likes","user"),
                           ("movie", "rev_rates", "user"),
                           ]
assert data["user"].num_nodes == 610
assert data["user"].num_features == 0
assert data["movie"].num_nodes == 9742
#assert data["movie"].num_features == 20
assert data["user", "rates", "movie"].num_edges == 100836
assert data["movie", "rev_rates", "user"].num_edges == 100836
#assert data["user", "likes", "user"].num_edges == 200

HeteroData(
  user={ node_id=[610] },
  movie={ node_id=[9742] },
  (user, rates, movie)={ edge_index=[2, 100836] },
  (user, likes, user)={ edge_index=[2, 199] },
  (movie, rev_rates, user)={ edge_index=[2, 100836] }
)


In [13]:
transform = T.RandomLinkSplit(
    num_val=0.1, 
    num_test=0.1,  
    disjoint_train_ratio=.3,  
    neg_sampling_ratio=2,  
    add_negative_train_samples=False,  
    edge_types=[("user", "rates", "movie"),("user","likes","user")],
    rev_edge_types=[("movie", "rev_rates", "user"),("user","likes","user")], 
)

train_data, val_data, test_data = transform(data)
print("Training data:")
print("==============")
print(train_data)
print()
print("Validation data:")
print("================")
print(val_data)


## Not needed. The data itself can work as the graph store. 
# train_graph_store = HeteroData()
# add_date_to_graph_store(train_graph_store, train_data)


assert train_data["user", "rates", "movie"].num_edges == 56469
assert train_data["user", "rates", "movie"].edge_label_index.size(1) == 24201
assert train_data["movie", "rev_rates", "user"].num_edges == 56469
# No negative edges added:
assert train_data["user", "rates", "movie"].edge_label.min() == 1
assert train_data["user", "rates", "movie"].edge_label.max() == 1

assert val_data["user", "rates", "movie"].num_edges == 80670
assert val_data["user", "rates", "movie"].edge_label_index.size(1) == 30249
assert val_data["movie", "rev_rates", "user"].num_edges == 80670
# Negative edges with ratio 2:1:
assert val_data["user", "rates", "movie"].edge_label.long().bincount().tolist() == [20166, 10083]

Training data:
HeteroData(
  user={ node_id=[610] },
  movie={ node_id=[9742] },
  (user, rates, movie)={
    edge_index=[2, 56469],
    edge_label=[24201],
    edge_label_index=[2, 24201]
  },
  (user, likes, user)={
    edge_index=[2, 113],
    edge_label=[48],
    edge_label_index=[2, 48]
  },
  (movie, rev_rates, user)={ edge_index=[2, 56469] }
)

Validation data:
HeteroData(
  user={ node_id=[610] },
  movie={ node_id=[9742] },
  (user, rates, movie)={
    edge_index=[2, 80670],
    edge_label=[30249],
    edge_label_index=[2, 30249]
  },
  (user, likes, user)={
    edge_index=[2, 161],
    edge_label=[57],
    edge_label_index=[2, 57]
  },
  (movie, rev_rates, user)={ edge_index=[2, 80670] }
)


In [16]:
# Define seed edges:
edge_label_index = train_data[("user", "rates", "movie")].edge_label_index
edge_label = train_data[("user", "rates", "movie")].edge_label

In [17]:
# Sample only with labels for user_rate_movies
train_loader = LinkNeighborLoader(
    data=(feature_store,train_data),  # <--- Changed to feature store, graph store
    num_neighbors=[20,10],  # Original [20,10]
    neg_sampling_ratio=2,  
    edge_label_index=(("user", "rates", "movie"), edge_label_index),
    edge_label=edge_label,
    batch_size=128, # Original 128
    shuffle=True,
)

# Inspect a sample:
sampled_data = next(iter(train_loader))

print("Sampled mini-batch:")
print("===================")
print(sampled_data)

assert sampled_data["user", "rates", "movie"].edge_label_index.size(1) == 3 * 128
assert sampled_data["user", "rates", "movie"].edge_label.min() == 0
assert sampled_data["user", "rates", "movie"].edge_label.max() == 1

Sampled mini-batch:
HeteroData(
  movie={
    num_nodes=2750,
    x=[2750, 20],
    node_id=[2750]
  },
  user={
    num_nodes=604,
    node_id=[604]
  },
  (user, rates, movie)={
    edge_index=[2, 17251],
    input_id=[128],
    edge_label_index=[2, 384],
    edge_label=[384]
  },
  (user, likes, user)={ edge_index=[2, 104] },
  (movie, rev_rates, user)={ edge_index=[2, 7784] }
)


In [26]:
user_edge_label_index = train_data[("user", "likes", "user")].edge_label_index
user_edge_label = train_data[("user", "likes", "user")].edge_label

In [27]:
# sample only for user_likes_user

# Sample only with labels for user_rate_movies
train_loader = LinkNeighborLoader(
    data=(feature_store,train_data),  # <--- Changed to feature store, graph store
    num_neighbors=[20,10],  # Original [20,10]
    neg_sampling_ratio=2,  
    edge_label_index=(("user","likes","user"),user_edge_label_index),
    edge_label=user_edge_label,
    batch_size=128, # Original 128
    shuffle=True,
)

# Inspect a sample:
sampled_data = next(iter(train_loader))

print("Sampled mini-batch:")
print("===================")
print(sampled_data)

# assert sampled_data["user", "rates", "movie"].edge_label_index.size(1) == 3 * 128
# assert sampled_data["user", "rates", "movie"].edge_label.min() == 0
# assert sampled_data["user", "rates", "movie"].edge_label.max() == 1


Sampled mini-batch:
HeteroData(
  movie={
    num_nodes=1831,
    x=[1831, 20],
    node_id=[1831]
  },
  user={
    num_nodes=604,
    node_id=[604]
  },
  (user, rates, movie)={ edge_index=[2, 13777] },
  (user, likes, user)={
    edge_index=[2, 81],
    input_id=[48],
    edge_label_index=[2, 144],
    edge_label=[144]
  },
  (movie, rev_rates, user)={ edge_index=[2, 4410] }
)
